### Import packages

In [1]:
import numpy as np 
import pandas as pd
import pickle
import scistreecna as scna

### Load example data

We begin by taking a quick look at the input data. Each row corresponds to a SNP site, and each column corresponds to a cell. In this example, the dataset contains **100 sites** and **60 cells**.

For each *(cell, site)* pair, the entry is a string in the format **`ref|alt|cn`**, where:

- **ref**: read count of the reference (wild-type) allele  
- **alt**: read count of the mutant allele  
- **cn**: observed copy number (either absolute copy number — recommended — or relative copy state)

Missing values should be encoded as:

- `.|.|cn` — if read counts are missing but copy number is available  
- `.|.|.` — if both read counts and copy number are missing
- `ref|alt|.` - if only copy number is missing

In [2]:
data = pd.read_csv('../examples/test_data_reads.csv', index_col=0)
data

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c50,c51,c52,c53,c54,c55,c56,c57,c58,c59
s0,11|0|2,20|1|1,25|0|2,9|0|2,36|0|2,17|1|2,13|0|1,27|1|2,7|0|2,21|0|2,...,23|0|2,5|0|2,22|1|2,13|0|2,34|1|2,15|0|2,0|0|2,23|0|2,12|0|2,18|0|2
s1,4|0|2,23|0|1,18|0|2,14|2|2,32|0|2,5|1|2,0|0|1,5|0|2,27|0|2,8|0|2,...,9|0|2,11|1|2,14|0|2,13|0|2,17|0|2,15|0|2,14|0|2,17|0|2,15|0|2,1|0|2
s2,13|0|2,26|0|2,20|0|2,6|0|2,17|0|2,12|0|2,34|0|2,21|0|2,24|0|2,24|0|2,...,11|0|1,12|2|2,13|0|2,22|0|2,21|1|2,20|0|2,2|1|2,21|1|2,13|0|2,16|0|2
s3,15|1|3,12|0|3,15|0|2,3|0|2,20|0|3,24|0|2,22|0|2,15|0|1,20|0|2,16|0|2,...,40|0|3,25|0|2,19|0|2,38|0|3,4|0|2,12|0|2,43|0|3,13|0|3,15|0|2,23|0|2
s4,22|0|2,15|0|2,15|5|2,19|0|2,9|0|2,17|7|2,24|0|2,19|0|2,10|0|2,5|8|2,...,22|0|2,6|4|2,32|0|2,4|0|2,26|1|2,6|5|2,10|7|2,23|0|2,25|0|2,10|0|2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s95,20|0|2,5|0|2,11|0|2,10|0|2,16|0|2,10|0|2,12|0|2,19|0|2,13|0|2,25|0|2,...,23|1|1,10|16|2,3|0|2,13|0|1,22|0|2,33|0|2,28|2|2,30|0|2,14|0|1,0|0|2
s96,20|0|1,23|0|2,13|0|2,32|0|2,33|0|3,0|0|2,9|0|2,8|0|2,24|0|2,9|0|2,...,21|0|2,5|0|2,6|0|2,16|0|2,5|0|2,30|0|2,5|0|2,0|0|2,2|0|2,12|0|2
s97,19|0|2,12|7|2,28|0|2,28|0|2,8|0|2,12|0|2,33|0|2,25|0|1,7|0|2,11|0|2,...,18|0|2,10|0|2,3|0|2,29|0|2,5|0|2,21|0|2,11|0|2,35|0|1,25|1|2,4|0|2
s98,30|0|2,17|0|2,19|0|2,13|0|2,26|1|2,1|0|2,4|0|2,9|0|2,35|0|2,0|0|2,...,20|0|2,16|1|2,22|0|2,8|0|2,26|0|2,9|0|2,14|1|2,29|0|2,39|0|3,29|0|2


Next, we load the data and parse it into a NumPy `ndarray`. We also extract the `cell_names` and `site_names` from the input table.  
>Note: Users may also prepare this data array directly from their own data, without explicitly creating a `.csv` file as shown above. ScisTreeCNA works directly with a `numpy.ndarray`.


In [3]:
# load .csv file 
reads, cell_names, site_names = scna.util.read_csv('../examples/test_data_reads.csv')

Now we are ready to run **scistreecna**. Before doing so, we need to configure several parameters, including allelic dropout (ADO), sequencing error rate, and copy-number noise.  
We must also specify the expected **minimum** and **maximum** copy numbers that best match the dataset. The minimum should be **greater than** 0, although the observed copy number in the input data may be 0.

In addition, `tree_batch_size` and `node_batch_size` control how the GPU parallelizes the computation. Setting these values too high may lead to out-of-memory (OOM) errors.


In [4]:
# run inference
scistreecna_tree, scistreecna_geno = scna.infer(reads,
                                              cell_names=cell_names,  # cell names
                                              ado=0.1,  # allelic dropout rate
                                              seq_error=0.01,   # sequencing error
                                              cn_noise=0.05,    # copy number noise
                                              cn_min=1, # minimum copy number (>=1) 
                                              cn_max=5, # maximum copy number
                                              tree_batch_size=128,  # number of trees evaluated together
                                              node_batch_size=256,  # number of nodes evaluated together 
                                              verbose=True)  # print logs

───────────────────────────────────────── ScisTreeCNA ──────────────────────────────────────────
                                      #Cell: 60 #Site: 100                                      
                   CN_MIN: 1 CN_MAX: 5 ADO: 0.1 SEQ_ERR: 0.01 CN_NOISE: 0.05                    
                           TREE_BATCH_SIZE: 128 NODE_BATCH_SIZE: 256                            
───────────────────────────────────────── Local Search ─────────────────────────────────────────
[09:46:14] [Iteration 0]   Likelihood: -12454.8754                                              
[09:46:16] [Iteration 1]   Likelihood: -12448.8588                                              
[09:46:17] [Iteration 2]   Likelihood: -12443.3693                                              
[09:46:18] [Iteration 3]   Likelihood: -12440.0679                                              
[09:46:19] [Iteration 4]   Likelihood: -12436.9292                                              
[09:46:20] [Iteration 5]   Lik

Finally, we can print the inferred tree and genotypes.

In [5]:
print(scistreecna_tree)

(((((((((((((((c11,c16),c1),c38),c19),((c27,c49),c37)),(c0,c53)),((c23,c57),c41)),((((c15,c35),c29),(c24,c48)),((c17,c30),c59))),((c4,c45),c10)),c50),(((((c31,c7),c36),c25),(c52,c6)),((c14,c28),c58))),((((((((((c32,c56),c5),c22),(c20,c43)),(c55,c9)),c2),((c21,c51),c18)),(c33,c46)),c47),(c13,c54))),(c12,c34)),c42),((((c26,c44),c8),c3),(c39,c40)));


In [6]:
print(scistreecna_geno)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


Since the ground truth for this dataset is available, we can load it and evaluate performance using two metrics:  
- **Tree accuracy**, measured as $1 - Robinson\_Foulds_{norm}$ 
- **Genotype accuracy**


In [7]:
# load ground truth
with open('../examples/test_data_tree.pkl', 'rb') as f:
    true_tree = pickle.load(f)
true_genotype = np.loadtxt('../examples/test_data_tg.txt', dtype=int)

In [8]:
tree_accuracy = scna.util.tree_accuracy(true_tree, scistreecna_tree)
geno_accuracy = scna.util.genotype_accuarcy(true_genotype, scistreecna_geno) 
print(f'Tree accuracy: {tree_accuracy}, Genotype accuracy: {geno_accuracy}')

Tree accuracy: 0.4827586206896552, Genotype accuracy: 0.9911666666666666


We can also run **scistree2** and compare the results with those from ScisTreeCNA.


In [9]:
scistree2tree, scistree2geno = scna.external.infer_scistree2_tree(reads, cell_names=cell_names)
scistree2_tree_accuracy = scna.util.tree_accuracy(true_tree, scistree2tree)
scistree2_geno_accuracy = scna.util.genotype_accuarcy(true_genotype, scistree2geno) 
print(f'Tree accuracy: {scistree2_tree_accuracy}, Genotype accuracy: {scistree2_geno_accuracy}')

Tree accuracy: 0.3793103448275862, Genotype accuracy: 0.9893333333333333


Actually, ScisTreeCNA integrates several external methods via `scn.external`, such as **CellPhy** and **DICE**.  
These methods require users to install them separately and provide the corresponding executable files.  
Here, we use **CellPhy** as an example.

In [10]:
cellphytree, cellphygeno = scna.external.infer_cellphy_tree(reads, executable='/home/haz19024/softwares/cellphy/cellphy.sh')
cellphy_tree_accuracy = scna.util.tree_accuracy(true_tree, cellphytree)
cellphy_geno_accuracy = scna.util.genotype_accuarcy(true_genotype, cellphygeno) 
print(f'Tree accuracy: {cellphy_tree_accuracy}, Genotype accuracy: {cellphy_geno_accuracy}')

Tree accuracy: 0.3620689655172414, Genotype accuracy: 0.8086666666666666
